In [130]:
# Reset all variables without asking for confirmation
%reset -f

# Importing Libraries

# Importing the 'euclidean' function from the 'scipy.spatial.distance' module
from scipy.spatial.distance import euclidean

# Importing 'perspective' and 'contours' modules from 'imutils'
from imutils import perspective
from imutils import contours

# Importing 'numpy' library and aliasing it as 'np'
import numpy as np

# Importing 'imutils' library
import imutils

# Importing 'cv2' (OpenCV) library for computer vision tasks
import cv2

# Define the path to the input image
img_path = "images/cenital.jpg"


In [131]:
# Read the image from the specified path
image = cv2.imread(img_path)

# Get the width, height, and number of color channels in the image
height, width, _ = image.shape

# Calculate the aspect ratio of the image
aspect_ratio = width / height

# Define the desired height for resizing
desired_height = 800

# Calculate the corresponding width to maintain the original aspect ratio
desired_width = int(desired_height * aspect_ratio)

# Resize the image to the desired dimensions
image = cv2.resize(image, (desired_width, desired_height))

# Convert the image from the BGR color space to the HSV color space
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Modify the saturation channel to adjust the saturation
saturation_factor = 3  # Adjust this factor to control saturation (e.g., 2.0 for doubling saturation)
hsv_image[:, :, 1] = np.clip(hsv_image[:, :, 1] * saturation_factor, 0, 255).astype(np.uint8)

# Convert the modified image back to the BGR color space (if needed)
result_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)

# Display the resulting image with adjusted saturation
cv2.imshow('Image with Adjusted Saturation', result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [132]:
# Apply the Canny edge detection algorithm to the result_image
edged = cv2.Canny(result_image, 120, 250)

# Dilate the edges to connect nearby edge points for better contour detection
edged = cv2.dilate(edged, None, iterations=4)

# Erode the edges to remove noise and make the contours smoother
edged = cv2.erode(edged, None, iterations=2)

# Display the original image with adjusted saturation
cv2.imshow('Image with Adjusted Saturation', result_image)
cv2.waitKey(0)

# Display the image with detected edges
cv2.imshow('Image with Edges', edged)
cv2.waitKey(0)

# Close all OpenCV windows
cv2.destroyAllWindows()


In [133]:
# Find contours in the grayscale image using the Canny edge-detected image
contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a copy of the original image to draw contours on
contour_image = image.copy()

# Draw the identified contours in green (color: (0, 255, 0)) with a thickness of 2 pixels
cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)

# Display the image with drawn contours
cv2.imshow('Image with Contours', contour_image)
cv2.waitKey(0)

# Close all OpenCV windows
cv2.destroyAllWindows()


In [134]:
# Create a copy of the original image to draw rotated rectangles on
image_with_rotated_rectangles_rot = image.copy()

# Define the conversion factor from pixels to centimeters
pixel_per_cm = 1.7

# Iterate through the identified contours
for cnt in contours:
    # Calculate the minimum area rectangle that encloses the contour
    box = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(box)
    box = np.array(box, dtype="int")
    box = perspective.order_points(box)
    (tl, tr, br, bl) = box

    # Draw the rotated rectangle on the image in black with a thickness of 1 pixel
    cv2.drawContours(image_with_rotated_rectangles_rot, [box.astype("int")], -1, (0, 0, 0), 1)

    # Calculate the midpoints of the horizontal and vertical sides of the rectangle
    mid_pt_horizontal = (tl[0] + int(abs(tr[0] - tl[0]) / 2), tl[1] + int(abs(tr[1] - tl[1]) / 2))
    mid_pt_vertical = (tr[0] + int(abs(tr[0] - br[0]) / 2), tr[1] + int(abs(tr[1] - br[1]) / 2))

    # Calculate the width and height of the rectangle in centimeters
    width_cm = euclidean(tl, tr) / pixel_per_cm
    height_cm = euclidean(tr, br) / pixel_per_cm

    # Add a text label indicating the width and height in centimeters at the bottom-right corner of the rectangle
    cv2.putText(image_with_rotated_rectangles_rot, f"[{width_cm:.1f}cm, {height_cm:.1f}cm]", (int(br[0]), int(br[1])),
                cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 0), 1)

# Save the image with drawn rectangles as a .jpg file
output_file = 'solution.jpg'
cv2.imwrite(output_file, image_with_rotated_rectangles_rot)

# Display the image with drawn rectangles
cv2.imshow('Image with Rotated Rectangles', image_with_rotated_rectangles_rot)
cv2.waitKey(0)

# Close all OpenCV windows
cv2.destroyAllWindows()
